# How to pass runtime values to tools

Sometimes we need to pass values to our tools at runtime. For example, you might want to give tools access to:

* graph state
* [shared memory](https://langchain-ai.github.io/langgraph/how-tos/memory/shared-state/) persisted between sessions

This type of stateful tools is useful when a tool's output is affected by past agent steps (e.g. if you're using a sub-agent as a tool, and want to pass the message history in to the sub-agent), or when a tool's input needs to be validated given context from past agent steps.

In this guide we'll demonstrate how to create tools that take agent state or LangGraph Store as input.

This is a special case of [passing runtime arguments to tools](https://python.langchain.com/docs/how_to/tool_runtime/), which you can learn about in the LangChain docs.

<div class="admonition tip">
    <p class="admonition-title">Prerequisites</p>
    <p>
        This guide assumes familiarity with the following:
        <ul>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/low_level/#state">
                    State
                </a>
            </li>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/#tool-calling-agent">
                    Tool-calling
                </a>
            </li>
        </ul>
    </p>
</div> 

## Setup

First we need to install the packages required

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-openai

Next, we need to set API keys for OpenAI (the chat model we will use).

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

OPENAI_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>    

## Pass graph state to tools

Let's first take a look at how to give our tools access to the graph.

### Define the tools

We'll want our tool to take graph state as an input, but we don't want the model to try to generate this input when calling the tool. We can use the `InjectedState` annotation to mark arguments as required graph state (or some field of graph state. These arguments will not be generated by the model. When using `ToolNode`, graph state will automatically be passed in to the relevant tools and arguments.

In this example we'll create a tool that returns Documents and then another tool that actually cites the Documents that justify a claim.

<div class="admonition note">
    <p class="admonition-title">Using Pydantic with LangChain</p>
    <p>
        This notebook uses Pydantic v2 <code>BaseModel</code>, which requires <code>langchain-core >= 0.3</code>. Using <code>langchain-core < 0.3</code> will result in errors due to mixing of Pydantic v1 and v2 <code>BaseModels</code>.
    </p>
</div>  

In [3]:
from typing import List, Tuple
from typing_extensions import Annotated

from langchain_core.documents import Document
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langgraph.prebuilt import InjectedState

from pydantic import BaseModel

docs = [
    Document(
        "FooBar company just raised 1 Billion dollars!",
        metadata={"source": "twitter"},
    ),
    Document(
        "FooBar company is now only hiring AI's", metadata={"source": "twitter"}
    ),
    Document(
        "FooBar company was founded in 2019", metadata={"source": "wikipedia"}
    ),
    Document(
        "FooBar company makes friendly robots", metadata={"source": "wikipedia"}
    ),
]

@tool(parse_docstring=True, response_format="content_and_artifact")
def get_context(question: List[str]) -> Tuple[str, List[Document]]:
    """Get context on the question.

    Args:
        question: The user question
    """
    # return constant dummy output   
    return "\n\n".join(doc.page_content for doc in docs), docs


@tool(parse_docstring=True, response_format="content_and_artifact")
def cite_context_sources(
    claim: str, state: Annotated[dict, InjectedState]
) -> Tuple[str, List[Document]]:
    """Cite which source a claim was based on.

    Args:
        claim: The claim that was made.
    """
    docs = []
    # We get the potentially cited docs from past ToolMessages in our state.
    for msg in state["messages"]:
        if isinstance(msg, ToolMessage) and msg.name == "get_context":
            for doc in msg.artifact:
                if isinstance(doc, Document):
                    docs.append(doc)
                elif isinstance(doc, dict):
                    docs.append(Document(**doc))

    class Cite(BaseModel):
        """Return the index(es) of the documents that justify the claim"""

        indexes: List[int]

    structured_model = model.with_structured_output(Cite)
    system = f"Which of the following documents best justifies the claim:\n\n{claim}"
    context = "\n\n".join(
        f"Document {i}:\n" + doc.page_content for i, doc in enumerate(docs)
    )
    citation = structured_model.invoke([("system", system), ("human", context)])
    cited_docs = [docs[i] for i in citation.indexes]
    sources = ", ".join(doc.metadata["source"] for doc in cited_docs)
    return sources, cited_docs

If we look at the input schemas for these tools, we'll see that `state` is still listed:

In [4]:
cite_context_sources.get_input_schema().schema()

{'description': 'Cite which source a claim was based on.',
 'properties': {'claim': {'description': 'The claim that was made.',
   'title': 'Claim',
   'type': 'string'},
  'state': {'title': 'State', 'type': 'object'}},
 'required': ['claim', 'state'],
 'title': 'cite_context_sources',
 'type': 'object'}

But if we look at the tool call schema, which is what is passed to the model for tool-calling, `state` has been removed:

In [5]:
cite_context_sources.tool_call_schema.schema()

{'description': 'Cite which source a claim was based on.',
 'properties': {'claim': {'description': 'The claim that was made.',
   'title': 'Claim',
   'type': 'string'}},
 'required': ['claim'],
 'title': 'cite_context_sources',
 'type': 'object'}

### Define the graph

In this example we will be using a [prebuilt ReAct agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/). We'll first need to define our model and a tool-calling node ([ToolNode](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.tool_node.ToolNode)):

In [6]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode, create_react_agent
from langgraph.checkpoint.memory import MemorySaver

model = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [get_context, cite_context_sources]

# ToolNode will automatically take care of injecting state into tools
tool_node = ToolNode(tools)

checkpointer = MemorySaver()
graph = create_react_agent(model, tools, checkpointer=checkpointer)

### Use it!

In [7]:
def pretty_print_stream(stream):
    for output in stream:
        # stream() yields dictionaries with output keyed by node name
        for key, value in output.items():
            print(f"Output from node '{key}':")
            print("---")
            for message in value["messages"]:
                message.pretty_print()
        print("\n---\n")

In [8]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage("what's the latest news about FooBar")]
config = {"configurable": {"thread_id": "1"}}
pretty_print_stream(graph.stream({"messages": messages}, config))

Output from node 'agent':
---
================================== Ai Message ==================================
Tool Calls:
  get_context (call_DDe4qFkNuWF9FeUcQVDqkuYF)
 Call ID: call_DDe4qFkNuWF9FeUcQVDqkuYF
  Args:
    question: ['latest news about FooBar']

---

Output from node 'tools':
---
================================= Tool Message =================================
Name: get_context

FooBar company just raised 1 Billion dollars!

FooBar company is now only hiring AI's

FooBar company was founded in 2019

FooBar company makes friendly robots

---

Output from node 'agent':
---
================================== Ai Message ==================================

The latest news about FooBar includes:

1. FooBar company has just raised 1 billion dollars.
2. FooBar company is now only hiring AI's.

---



In [9]:
messages = [HumanMessage("where did you get this information?")]
pretty_print_stream(graph.stream({"messages": messages}, config))

Output from node 'agent':
---
================================== Ai Message ==================================
Tool Calls:
  cite_context_sources (call_JoXpwqa4Ybq86QbC05ZVGUoY)
 Call ID: call_JoXpwqa4Ybq86QbC05ZVGUoY
  Args:
    claim: FooBar company just raised 1 Billion dollars!
  cite_context_sources (call_v8zaP5YwjrSHodLjpQnnCwiM)
 Call ID: call_v8zaP5YwjrSHodLjpQnnCwiM
  Args:
    claim: FooBar company is now only hiring AI's

---

Output from node 'tools':
---
================================= Tool Message =================================
Name: cite_context_sources

twitter
================================= Tool Message =================================
Name: cite_context_sources

twitter

---

Output from node 'agent':
---
================================== Ai Message ==================================

The information about FooBar company raising 1 billion dollars and only hiring AI's was sourced from Twitter.

---



## Pass shared memory (store) to the graph

You might also want to give tools access to memory that is shared across multiple conversations or users. We can do it by passing LangGraph [Store](https://langchain-ai.github.io/langgraph/how-tos/memory/shared-state/) to the tools using a different annotation -- `InjectedStore`.

Let's modify our example to save the documents in an in-memory store and retrieve them using `get_context` tool. We'll also make the documents accessible based on a user ID, so that some documents are only visible to certain users. The tool will then use the `user_id` provided in the [config](https://langchain-ai.github.io/langgraph/how-tos/pass-config-to-tools/) to retrieve a correct set of documents.

<div class="admonition note">
    <p class="admonition-title">Note</p>
    <list>
        <li>
        Support for <code>Store</code> API and <code>InjectedStore</code> used in this notebook was added in LangGraph <code>v0.2.34</code>.
        </li>
        <li>
        <code>InjectedStore</code> annotation requires <code>langchain-core >= 0.3.8</code>
        </li>
    <list>
</div>

In [ ]:
<div class="admonition warning">
    <p class="admonition-title">Note</p>
    <p>
    
    </p>
</div>

In [10]:
from langgraph.store.memory import InMemoryStore

doc_store = InMemoryStore()

for doc_id, doc in enumerate(docs[:2]):
    user_id = "1"
    namespace = ("documents", user_id)
    doc_store.put(namespace, doc_id, {"doc": doc})


for doc_id, doc in enumerate(docs[2:]):
    user_id = "2"
    namespace = ("documents", user_id)
    doc_store.put(namespace, doc_id, {"doc": doc})

### Define the tools

In [11]:
from langgraph.store.base import BaseStore
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt import InjectedStore


@tool(parse_docstring=True, response_format="content_and_artifact")
def get_context(
    question: List[str],
    config: RunnableConfig,
    store: Annotated[BaseStore, InjectedStore()]
) -> Tuple[str, List[Document]]:
    """Get context on the question.

    Args:
        question: The user question
    """
    # return constant dummy output   
    user_id = config.get("configurable", {}).get("user_id")
    docs = [item.value["doc"] for item in store.search(("documents", user_id))]
    return "\n\n".join(doc.page_content for doc in docs), docs

We can also verify that the tool-calling model will ignore `store` arg of `get_context` tool:

In [12]:
get_context.tool_call_schema.schema()

{'description': 'Get context on the question.',
 'properties': {'question': {'description': 'The user question',
   'items': {'type': 'string'},
   'title': 'Question',
   'type': 'array'}},
 'required': ['question'],
 'title': 'get_context',
 'type': 'object'}

### Define the graph

Let's update our ReAct agent:

In [13]:
tools = [get_context, cite_context_sources]

# ToolNode will automatically take care of injecting Store into tools
tool_node = ToolNode(tools)

checkpointer = MemorySaver()
# NOTE: we need to pass our store to `create_react_agent` to make sure our graph is aware of 
graph = create_react_agent(model, tools, checkpointer=checkpointer, store=doc_store)

### Use it!

Let's try running our graph with a `"user_id"` in the config.

In [14]:
messages = [HumanMessage("what's the latest news about FooBar")]
config = {"configurable": {"thread_id": "1", "user_id": "1"}}
pretty_print_stream(graph.stream({"messages": messages}, config))

Output from node 'agent':
---
================================== Ai Message ==================================
Tool Calls:
  get_context (call_oaUzgWpNUJrmi9lON6uofCIB)
 Call ID: call_oaUzgWpNUJrmi9lON6uofCIB
  Args:
    question: ['latest news about FooBar']

---

Output from node 'tools':
---
================================= Tool Message =================================
Name: get_context

FooBar company just raised 1 Billion dollars!

FooBar company is now only hiring AI's

---

Output from node 'agent':
---
================================== Ai Message ==================================
Tool Calls:
  cite_context_sources (call_6b0c1zyVBWF6dG60IAL9NLjS)
 Call ID: call_6b0c1zyVBWF6dG60IAL9NLjS
  Args:
    claim: FooBar company just raised 1 Billion dollars!
  cite_context_sources (call_hby5NYD1rdWW0I7jvdXJIJHE)
 Call ID: call_hby5NYD1rdWW0I7jvdXJIJHE
  Args:
    claim: FooBar company is now only hiring AI's

---

Output from node 'tools':
---
================================= Tool M

We can see that the tool only retrieved the correct documents for user "1" when looking up the information in the store. Let's now try it again for a different user:

In [15]:
messages = [HumanMessage("what's the latest news about FooBar")]
config = {"configurable": {"thread_id": "2", "user_id": "2"}}
pretty_print_stream(graph.stream({"messages": messages}, config))

Output from node 'agent':
---
================================== Ai Message ==================================
Tool Calls:
  get_context (call_VFWTHaqHWH5W22zMUyEKNnLL)
 Call ID: call_VFWTHaqHWH5W22zMUyEKNnLL
  Args:
    question: ['latest news about FooBar']

---

Output from node 'tools':
---
================================= Tool Message =================================
Name: get_context

FooBar company was founded in 2019

FooBar company makes friendly robots

---

Output from node 'agent':
---
================================== Ai Message ==================================

FooBar is a company that was founded in 2019 and specializes in making friendly robots. If you need more specific or recent news, please let me know!

---



We can see that the tool pulled in a different set of documents this time.